### Extract from OSM
The purpose of this script is to show how to go from a .osm.pbf file to a network format that GOSTnets will accept. 
Additional modifications in this script include: clipping the roads to an input polygon (here, kinshasa.shp), and slicing the roads DataFrame by a list of accepted road types, as tagged in OSM (see accepted_road_types list). 

Before running this, make sure to register your sparkly new environment. By typing: 
conda activate myenv 

python -m ipykernel install --user --name [py36] 


In [1]:
import sys
#!conda install --yes --prefix {sys.prefix} boltons

import geopandas as gpd
import pandas as pd
import os, sys
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
codepath = r"C:\Users\wb495141\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets"

sys.path.append(codepath)
import GOSTnet as gn
import LoadOSM as losm
import importlib

Define filepaths. Since we just need some randome baseline OSM data to which we can add a new road shapefile, we use uzbekistan-latest.osm.pbf. But this could be anything.

In [2]:
pth = r'C:\Users\wb495141\OneDrive - WBG\poverty\chad\input'
pth2 = r'C:\Users\wb495141\OneDrive - WBG\poverty\chad\temp'
fil = r'uzbekistan-latest.osm.pbf'
f = os.path.join(pth, fil)

Using the loadOSM library, pass the filepath of the .osm.pbf to initiate a OSM_to_network class

In [3]:
road_osm = losm.OSM_to_network(f)

AttributeError: 'NoneType' object has no attribute 'ExecuteSQL'

This generates a property of the class called 'roads_raw'. This is a raw dataframe of the highway objects, extracted from the .osm.pbf. This is the starting point for our network

In [14]:
actual_chad = gpd.read_file(os.path.join(pth, 'road.shp'))

In [16]:
actual_chad['infra_type'] = 'road'
actual_chad['osm_id'] = actual_chad.index
actual_chad = actual_chad[['osm_id','infra_type','geometry','LENGTH']]
actual_chad

,osm_id,infra_type,geometry,LENGTH
0,0,road,LINESTRING (18.33049237777778 21.7441127197520...,60.760
1,1,road,LINESTRING (18.02387461777777 21.7206599834421...,33.049
2,2,road,LINESTRING (19.21814005777777 21.5685337272900...,39.488
3,3,road,LINESTRING (19.21716337777778 21.5667178875737...,0.225
4,4,road,LINESTRING (19.30348057777778 21.4929646090796...,41.913
5,5,road,LINESTRING (19.30348057777778 21.4929646090796...,12.239
6,6,road,LINESTRING (18.33049237777778 21.7441127197520...,114.426
7,7,road,LINESTRING (16.99947013777778 21.3571747001693...,176.753
8,8,road,LINESTRING (17.12890453777777 21.3374050332295...,85.986
9,9,road,LINESTRING (16.99947013777778 21.3571747001693...,14.885


In [6]:
road_osm.roads_raw = actual_chad

In [7]:
road_osm.roads_raw.infra_type.value_counts()

road    2226
Name: infra_type, dtype: int64

Note that kin.generateRoadsGDF runs only if input roadshapefile is projected in WGS84

In [8]:
road_osm.generateRoadsGDF(verbose = False)

With initialReadIn(), we transform this to a graph object

In [9]:
road_osm.initialReadIn()

We save this graph object down to file using gn.save()

In [10]:
gn.save(road_osm.network,'chad_G_unclean',pth2)